In [35]:
# choose execl 
def choose_file():
    
    return 'FFG-November-Options-Activity' + '.xlsx'
    #return 'SCT 04-30-18' + '.xlsx'

In [5]:
# input shares dates
def get_date():
    #return datetime.date(2018, 01, 24)  
    return datetime.datetime.now()


In [6]:
# read xls into pandas table
def create_table(file_name):
    tables_list = pd.read_excel( file_name, sheetname=None)
    
    # get last worksheet from list
    table = tables_list[list(tables_list.keys())[-1]]
    
    # clear nan rows
    table = table[pd.isnull(table.SYMBL) == False]
    
    # extract existing found-codes
    symbl = table.SYMBL.dropna().unique()
    
    return table, symbl  # return table and list of found market symbols


In [47]:
shares_panel = pdr.get_data_yahoo(list(symbl), start, end)


[*********************100%***********************]  9 of 9 downloaded


In [48]:
shares_panel.to_frame()

,
major,minor


In [49]:
start, end

(datetime.datetime(2018, 5, 9, 0, 0), datetime.datetime(2018, 5, 10, 0, 0))

In [39]:
# read and preprocess shares from yahoo on current date
def read_shares(symbl, start, end):

    #start = end = datetime.datetime.now()
    start = end = get_date()

    # вставка фикса для yahoo
    from pandas_datareader import data as pdr

    import fix_yahoo_finance as yf
    yf.pdr_override()  # <== that's all it takes :-)

    # download dataframe
    shares_panel = pdr.get_data_yahoo(list(symbl), start, end)
    # конец вставки

    #shares_panel = web.DataReader(symbl, 'iex', start, end)

    # convert Panel to the Multyindexed frame
    shares = shares_panel.to_frame()

    # remove yesterday (start_date -1) from multyindex df
    yesterday = start - timedelta(days=1)
    #print (yesterday)
    #print (shares)
    shares.drop(yesterday, level=0, axis=0, inplace=True)
    # print shares

    # remove index level 0 (data)
    shares.index = shares.index.droplevel(0)

    # convert index into column
    shares.reset_index(inplace=True)

    shares.rename(columns={'minor': 'symbl'}, inplace=True)

    # insert column
    shares['Current Date'] = start

    # remove column
    #shares.drop(['Volume'], axis=1, inplace=True)

    return shares

In [30]:
# create output xls from table and shares dataframes
def create_output(table, shares):
    # create table
    table = pd.merge(left=table, right=shares, left_on='SYMBL', right_on='symbl')
    
    # delete 2-nd `symbol`
    table.drop('symbl', axis=1, inplace=True)
    
    # reformat date columns to m/d/y
    # table['Current Date'] = table['Current Date'].map(lambda x: x.strftime(format='%m/%d/%Y'))
    # table['EXP DATE'] = table['EXP DATE'].strftime('%m/%d/%Y')
    # table['EXP DATE'] = table['EXP DATE'].dt.strftime('%m/%d/%Y')
    
    # reformat prices to 2 dec.digits
    round_2 = lambda x: round(x, 2)
    table.Open = table.Open.map(round_2)
    table.Low = table.Low.map(round_2)
    table.High = table.High.map(round_2)
    table.Close = table.Close.map(round_2)
    table['Adj Close'] = table['Adj Close'].map(round_2)
    
    # reorder columns
    order = list(table.columns)[:-6]  # exept new columns
    new_order = order[:len(order)] + ['Current Date', 'Open', 'High', 'Low', 'Close', 'Adj Close']
    new_table = table[new_order]
    
    # reorder SEQ column through ass
    new_table.SEQ = range(1, new_table.SEQ.shape[0] + 1)
    
    # write new xls table
    new_table.to_excel('result_' + file_name, index=False)


### Start here

In [63]:
# main part
import os
import pandas as pd
import pandas_datareader.data as web
from datetime import date
from datetime import datetime
import datetime
from datetime import timedelta

In [64]:
file_name = choose_file()
start_date = end_date = get_date()
table, symbl = create_table(file_name)  # create pandas df
print ('\nwork tables created...')
shares = read_shares(symbl, start_date, end_date)  # deliver shares from yahoo
print ('shares delivered...')
create_output(table, shares)  # create, reformat and write output xls
print ('output file created')



work tables created...
[*********************100%***********************]  9 of 9 downloaded
shares delivered...
output file created


In [12]:
import pandas_datareader.data as web
from datetime import datetime
start = datetime(2018, 5, 9)
end = datetime(2018, 5, 10)

In [13]:
f = web.DataReader('F', 'morningstar', start, end)

NotImplementedError: data_source='morningstar' is not implemented

In [66]:
f

Close   High    Low   Open    Volume
Symbol Date                                            
F      2018-05-09  11.06  11.28  11.02  11.21  43731958
       2018-05-10  11.21  11.25  11.02  11.04  31519940

In [187]:
shares

,symbl,Adj Close,Close,High,Low,Open,Current Date
0,ABT,61.720001,61.720001,64.160004,60.740002,61.000000,2018-01-24
1,BX,36.080002,36.080002,36.330002,35.680000,36.250000,2018-01-24
2,CMCSA,42.990002,42.990002,44.000000,41.570000,42.700001,2018-01-24
3,CVX,131.389999,131.389999,132.669998,131.070007,131.380005,2018-01-24
4,MDT,85.760002,85.760002,86.129997,85.160004,85.830002,2018-01-24
5,MS,57.240002,57.240002,57.730000,56.900002,57.209999,2018-01-24
6,PM,110.599998,110.599998,110.889999,109.559998,109.559998,2018-01-24
7,SUN,32.540001,32.540001,33.040001,32.419998,32.529999,2018-01-24
8,UN,58.240002,58.240002,58.320000,57.990002,58.279999,2018-01-24


In [14]:
symbl

array([u'CMCSA', u'SUN', u'CVX', u'MDT', u'ABT', u'BX', u'PM', u'UN', u'MS'], dtype=object)

In [163]:
panel = web.DataReader(symbl, 'yahoo', start_date, end_date)

In [114]:
aaa.index.levels[0][0]

Timestamp('2018-01-24 00:00:00')

In [177]:
aaa.index.names

FrozenList([u'Date', u'minor'])

In [188]:
aaa.drop(start_date - timedelta(days=1) , level=0, axis=0)

Adj Close       Close        High         Low        Open  \
Date       minor                                                               
2018-01-24 ABT     61.720001   61.720001   64.160004   60.740002   61.000000   
           BX      36.080002   36.080002   36.330002   35.680000   36.250000   
           CMCSA   42.990002   42.990002   44.000000   41.570000   42.700001   
           CVX    131.389999  131.389999  132.669998  131.070007  131.380005   
           MDT     85.760002   85.760002   86.129997   85.160004   85.830002   
           MS      57.240002   57.240002   57.730000   56.900002   57.209999   
           PM     110.599998  110.599998  110.889999  109.559998  109.559998   
           SUN     32.540001   32.540001   33.040001   32.419998   32.529999   
           UN      58.240002   58.240002   58.320000   57.990002   58.279999   

                      Volume  
Date       minor              
2018-01-24 ABT    11704900.0  
           BX      5629000.0  
           CMCSA  48918800.0  
           CVX     6868200.0  
           MDT     5680200.0  
           MS      8403600.0  
           PM      4404100.0  
           SUN      490200.0  
           UN      1281200.0

In [149]:
from datetime import timedelta

In [151]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)

In [173]:
start_date - timedelta(days=1)

datetime.date(2018, 1, 23)

In [61]:
from pandas_datareader import data as pdr

import fix_yahoo_finance as yf
yf.pdr_override() # <== that's all it takes :-)

# download dataframe
data = pdr.get_data_yahoo(list(symbl), start="2017-01-01", end="2017-04-30")


[*********************100%***********************]  9 of 9 downloaded


In [62]:
data.to_frame()

Open        High         Low       Close   Adj Close  \
Date       minor                                                               
2017-01-03 ABT     38.630001   39.080002   38.340000   39.049999   37.800980   
           BX      27.500000   28.219999   27.350000   28.120001   25.219246   
           CMCSA   34.834999   34.945000   34.115002   34.525002   33.772381   
           CVX    118.379997  119.000000  116.589996  117.849998  112.229385   
           MDT     70.250000   71.029999   69.349998   70.610001   68.671280   
           MS      43.090000   43.630001   42.450001   43.049999   41.842842   
           PM      91.790001   92.000000   90.919998   91.230003   86.964432   
           SUN     27.000000   27.160000   26.900000   27.000000   22.848328   
           UN      40.430000   40.650002   40.349998   40.610001   38.737759   
2017-01-04 ABT     39.060001   39.730000   39.060001   39.360001   38.101067   
           BX      28.500000   30.480000   28.500000   29.799999   26.725943   
           CMCSA   34.755001   35.130001   34.590000   34.935001   34.173443   
           CVX    118.410004  118.650002  117.599998  117.820000  112.200821   
           MDT     70.900002   71.660004   70.820000   71.269997   69.313148   
           MS      43.299999   43.820000   43.040001   43.619999   42.396854   
           PM      90.529999   91.120003   90.120003   90.419998   86.192314   
           SUN     27.010000   27.830000   27.010000   27.450001   23.229134   
           UN      40.840000   40.980000   40.750000   40.919998   39.033466   
2017-01-05 ABT     39.240002   39.750000   38.980000   39.700001   38.430191   
           BX      29.850000   30.629999   29.700001   30.420000   27.281988   
           CMCSA   34.794998   35.130001   34.700001   35.075001   34.310394   
           CVX    118.000000  118.480003  116.720001  117.309998  111.715141   
           MDT     71.480003   71.510002   70.589996   70.830002   68.885239   
           MS      43.529999   43.630001   42.189999   43.220001   42.008072   
           PM      90.690002   91.760002   90.529999   91.129997   86.869110   
           SUN     27.590000   27.760000   27.180000   27.200001   23.017574   
           UN      41.090000   41.380001   41.009998   41.290001   39.386417   
2017-01-06 ABT     39.700001   40.990002   39.660000   40.779999   39.475647   
           BX      30.700001   30.910000   30.209999   30.510000   27.362703   
           CMCSA   35.105000   35.270000   34.910000   35.134998   34.369080   
...                      ...         ...         ...         ...         ...   
2017-04-25 PM     112.379997  113.449997  112.000000  113.089996  108.794769   
           SUN     30.450001   31.020000   30.200001   30.959999   26.951563   
           UN      52.470001   52.560001   52.310001   52.459999   50.463692   
2017-04-26 ABT     43.930000   44.150002   43.709999   43.799999   42.933933   
           BX      31.080000   31.440001   30.969999   31.270000   28.473787   
           CMCSA   38.680000   39.180000   38.500000   38.790001   38.104061   
           CVX    106.400002  107.480003  105.970001  106.080002  101.987694   
           MDT     82.239998   82.970001   82.070000   82.139999   80.310402   
           MS      44.169998   44.860001   44.139999   44.369999   43.322655   
           PM     112.860001  113.000000  111.089996  111.150002  106.928452   
           SUN     30.969999   31.200001   30.750000   31.000000   26.986383   
           UN      52.310001   52.349998   51.959999   51.980000   50.001965   
2017-04-27 ABT     43.959999   44.009998   43.599998   43.650002   42.786903   
           BX      30.450001   31.030001   30.150000   30.969999   29.007669   
           CMCSA   39.549999   40.619999   39.189999   39.590000   38.889912   
           CVX    105.959999  105.980003  104.669998  105.470001  101.401237   
           MDT     82.220001   83.500000   82.120003   83.129997   81.278343   
           MS      44.16

In [56]:
data.columns

AttributeError: 'Panel' object has no attribute 'columns'